# Computing Setbacks Exclusions

**Prerequisites:**

- Required: [GeoTIFFs to reV HDF5 Files](./02_layered_h5_tutorial.ipynb)
- Recommended: [Working with GeoTIFFs](./01_geotiff_tutorial.ipynb)

----

## Introduction

In the context of renewable energy siting, setbacks represent the minimum distance a generator can be located from a particular feature. In `reV`, we model setbacks as exclusion layers computed for each feature type (i.e. setbacks from rail, roads, transmission, structures, etc).

This tutorial will demonstrate how to calculate these setbacks from one of these features (railroads) using the [`reVX` setbacks utility](https://nrel.github.io/reVX/misc/examples.setbacks.html), especially in the context of HDF5 file inputs for [reV](https://github.com/NREL/reV) modeling.

Let's get started! 

Let's start with a few common imports:

In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import plotly.express as px

from reVX.handlers.layered_h5 import LayeredH5
from reVX.handlers.geotiff import Geotiff
from reVX.setbacks import SETBACKS
from reVX.setbacks.regulations import (
    SetbackRegulations, WindSetbackRegulations
)

## Downloading the data

For this tutorial, the only extra data we will need is the GeoPackage of existing railroads. We will "download" this file using `geopandas`. 

We'll start by defining the local file path destinations:

In [2]:
# Define paths to data and h5 file
H5_PATH = "example.h5"  # Built in prerequisite notebook
FEATURE_URL = "https://www2.census.gov/geo/tiger/TIGER2023/RAILS/tl_2023_us_rails.zip"
FEATURES_FPATH = "hsip_2012_rail.gpkg"
REGULATION_FPATH = "demo_regulations.csv"

Now we can use `geopandas` to read the file URL directly and then save it locally as a GeoPackage:

In [3]:
gdf = gpd.read_file(FEATURE_URL)
gdf.to_file(FEATURES_FPATH, driver="GPKG", index=False)

## Computing Setbacks

To compute setbacks from a particular feature, we first have to select the correct setback computation class. To do thins, we can select from one of the following:

In [ ]:
list(SETBACKS)

In this tutorial, we will focus on railroad setbacks, so we will select our setback calculator class to be `"rail"`:

In [5]:
setback_calculator = SETBACKS["rail"]

The main method we will be using is `run()`. To get more details on the inputs required for this method, we can run

In [ ]:
help(setback_calculator.run)

There are several approaches we can use to compute setback exclusions from railroads. 
We will explore the following in this tutorial:

1) Fixed-distance setbacks
2) Turbine tip-height setbacks
3) Multiplier setbacks
4) Partial setbacks
5) Local county ordinance setbacks

### Fixed-distance Setbacks

Fixed-distance setbacks are the most straightforward type of setbacks to compute. For this approach, we will just select a minimum length (in m) that a generator has to be placed away from the feature (i.e. railroad in our case). Let's suppose we was to compute two different fixed setback values: 100 meters and 1,000 meters. For each setback value, we have to create a `SetbackRegulation` instance, and then use it to compute the setbacks, like so:

In [7]:
for setback_distance in [100, 1000]:
    regulations = SetbackRegulations(
        base_setback_dist=setback_distance, multiplier=1
    )
    out_fn = f"setbacks_rail_{setback_distance}m.tif"
    setback_calculator.run(
        excl_fpath=H5_PATH,
        features_path=FEATURES_FPATH,
        out_fn=out_fn,
        regulations=regulations,
    )

<div class="alert alert-block alert-info">
<b>Note:</b> The <code class="python">SetbackRegulation</code> class requires a multiplier as input. We will go over the use cases of this input below, but you can always set it to `1` to compute a static setback distance.
</div>

We can verify that the setbacks were computed properly by plotting the results:

In [ ]:
def title_replace(a):
    a.update(
        text=(
            a.text
            .replace("facet_col=0", "100m Railroad Setbacks")
            .replace("facet_col=1", "1000m Railroad Setbacks")
        )
    )


all_layers = []
for setback_distance in [100, 1000]:
    with Geotiff(f"setbacks_rail_{setback_distance}m.tif") as geo:
        all_layers.append(geo.values[0])

fig = px.imshow(
    np.r_[all_layers],
    facet_col=0,
    facet_col_wrap=2,
    height=350,
    width=700,
    color_continuous_scale="BuPu",
)
fig.update_layout(font={"size": 16})
fig.update_xaxes(visible=False)
fig.update_yaxes(visible=False)
fig.for_each_annotation(title_replace)
fig.show()

### Turbine tip-height setbacks

If you are computing setbacks for wind turbine generators, you can specify the regulations using the `WindSetbackRegulations` class. 
This class records the turbine hub height and rotor diameter and computes the base setback distance to be the turbine tip height. It can also help compute local ordinance setback distances, which often mix the three turbine dimensions (hub height, rotor diameter, and max tip height) in their regulation text.

In [ ]:
hh, rd = 115, 50
regulations = WindSetbackRegulations(
    hub_height=hh,
    rotor_diameter=rd,
    multiplier=1,
)
# Base setback distance should be HH (115) + RD/2 (25) = 140
regulations.base_setback_dist

In [ ]:
out_fn = f"setbacks_rail_{hh}hh_{rd}rd.tif"
setback_calculator.run(
    excl_fpath=H5_PATH,
    features_path=FEATURES_FPATH,
    out_fn=out_fn,
    regulations=regulations,
)

### Multiplier setbacks

Local ordinances often give the setback value as a multiplier applied to a dimension, such as the height of the generator. For this reason, the regulation classes in `reVX` have a multiplier input to easily match the ordinance specification. For example, suppose you need to compute a 1.7 max tip height setback for the turbine defined in a previous section:

In [ ]:
regulations = WindSetbackRegulations(
    hub_height=hh,
    rotor_diameter=rd,
    multiplier=1.7,
)
out_fn = f"setbacks_rail_{hh}hh_{rd}rd_1.7.tif"
setback_calculator.run(
    excl_fpath=H5_PATH,
    features_path=FEATURES_FPATH,
    out_fn=out_fn,
    regulations=regulations,
)

We can check that this indeed computed a larger setback than before:

In [ ]:
def title_replace(a):
    a.update(
        text=(
            a.text
            .replace("facet_col=0", "1x Tip Height Setbacks")
            .replace("facet_col=1", "1.7x Tip Height Setbacks")
        )
    )


all_layers = []
with Geotiff(f"setbacks_rail_{hh}hh_{rd}rd.tif") as geo:
    all_layers.append(geo.values[0])
with Geotiff(f"setbacks_rail_{hh}hh_{rd}rd_1.7.tif") as geo:
    all_layers.append(geo.values[0])

fig = px.imshow(
    np.r_[all_layers],
    facet_col=0,
    facet_col_wrap=2,
    height=350,
    width=700,
    color_continuous_scale="BuPu",
)
fig.update_layout(font={"size": 16})
fig.update_xaxes(visible=False)
fig.update_yaxes(visible=False)
fig.for_each_annotation(title_replace)
fig.show()

### Partial setbacks

In some cases, the setback distance we want to compute is smaller than the resolution of the underlying raster. If we computed boolean exclusions (i.e. pixel = 1 for excluded and 0 for included), we would overestimate the amount of land excluded due to this setback. Instead, what we can do is compute a _partial inclusion_ raster, where values represent the fraction of the pixel that is *included* (i.e. 0 = fully excluded, 1 = fully included, 0.5 = half excluded, half included, etc.). To do this, we can specify a `weights_calculation_upscale_factor` in the input to the `run` function. This value should be an integer that will ultimately determine how fine the fractional output values are. Note that setting this value too large may cause your program to run out of memory. A good run of thumb is to set it to some value <= 10.

In [ ]:
setback_distance = 30  # this is 1/3 of the side length of out 90m cells
regulations = SetbackRegulations(
    base_setback_dist=setback_distance, multiplier=1
)
out_fn = f"setbacks_rail_{setback_distance}m.tif"
setback_calculator.run(
    excl_fpath=H5_PATH,
    features_path=FEATURES_FPATH,
    out_fn=out_fn,
    regulations=regulations,
    weights_calculation_upscale_factor=5,
)

We can check the output values like so:

In [ ]:
with Geotiff("setbacks_rail_30m.tif") as geo:
    data = geo.values[0]

unique_vals = ", ".join([f"{v:.2f}" for v in sorted(np.unique(data))])

print(f"Unique inclusion values:\n{unique_vals}")

fig = px.imshow(
    data,
    height=700,
    width=700,
    color_continuous_scale="BuPu",
    title="30m Railroad Setbacks"
)
fig.update_layout(font={"size": 16})
fig.update_xaxes(visible=False)
fig.update_yaxes(visible=False)
fig.show()

### Local county ordinance setbacks

So far we have only explored computing setbacks everywhere. However, sometimes it is valuable to be able to compute setbacks based on local county ordinances. For this, we need two extra pieces of information: 1) The local county ordinance setback values formatted into a compatible CSV file and 2) a `FIPS` layer that maps region s in our base raster to county FIPS values. 

For the purposes of this demonstration, we will create our own fictional FIPS layer. In practice, you could create the FIPS layer by rasterizing a FIPS vector file onto your region.

Let's start by breaking our region into four quadrants, and setting our demo FIPS values to be 1-4 in those quadrants:

In [ ]:
h5 = LayeredH5(H5_PATH)
fips_layer = np.ones(h5.shape)
width, height = fips_layer.shape
fips_layer[:, height // 2:] += 1
fips_layer[width // 2:] += 2
fips_layer

Great! Now we have to write this data to our HDF5 file. The layer _must_ go in under the name `cnty_fips`:

In [ ]:
h5.write_layer_to_h5(
    values=fips_layer, layer_name="cnty_fips", profile=h5.profile
)
h5.layers

The other extra piece of information we will need is a CSV file containing the local regulations. We will create a sample regulation file for the purposes of this example:

In [20]:
reg_1 = {
    "Feature Type": "railroads",
    "Feature Subtype": None,
    "Value Type": "Hub-height Multiplier",
    "Value": 3,
    "FIPS": 1
}
reg_2 = {
    "Feature Type": "railroads",
    "Feature Subtype": None,
    "Value Type": "Max-tip Height Multiplier",
    "Value": 7,
    "FIPS": 2
}
reg_3 = {
    "Feature Type": "railroads",
    "Feature Subtype": None,
    "Value Type": "Meters",
    "Value": 2000,
    "FIPS": 3
}
pd.DataFrame([reg_1, reg_2, reg_3]).to_csv(REGULATION_FPATH, index=False)

Now we can compute the setback values like before, with one extra input:

In [ ]:
regulations = WindSetbackRegulations(
    hub_height=hh,
    rotor_diameter=rd,
    regulations_fpath=REGULATION_FPATH
)
out_fn = f"setbacks_rail_{hh}hh_{rd}rd_regulations.tif"
setback_calculator.run(
    excl_fpath=H5_PATH,
    features_path=FEATURES_FPATH,
    out_fn=out_fn,
    regulations=regulations,
)

We can plot the result to verify that the local setbacks have indeed been applied:

In [ ]:
with Geotiff(out_fn) as geo:
    data = geo.values[0]

fig = px.imshow(
    data,
    height=700,
    width=700,
    color_continuous_scale="BuPu",
    title="Local County Railroad Setbacks"
)
fig.update_layout(font={"size": 16})
fig.update_xaxes(visible=False)
fig.update_yaxes(visible=False)
fig.show()

## Computing Setbacks via CLI

Alternatively, the command line can be used to compute setbacks for one or more features.

First, we need to construct a JSON config file (e.g. `setback.json`) that contains the parameters we discussed above:

```json
{
    "excl_fpath": "example.h5",
    "regulations_fpath": "demo_regulations.csv",
    "hub_height": 115,
    "rotor_diameter": 50,
    "features": {
        "rail": "hsip_2012_rail.gpkg"
    }
}
```

Then we can run the following command: 

```bash
$ setbacks compute --config_file setback.json
```

A more detailed guide to computing setbacks from the command line can be found [here](https://nrel.github.io/reVX/misc/examples.setbacks.html).

## Conclusion
In this tutorial, we have walked through the basic steps of computing setback exclusions. You should now be able to compute:
- Fixed-distance setbacks
- Turbine tip-height setbacks
- Multiplier setbacks
- Partial setbacks
- Local county ordinance setbacks